# <center><u> Final LSTM/GRU Layout - Adri and Elian

In [31]:
#Import Libraries and Packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
#Access Credentials
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")

In [33]:
conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

In [34]:
query="""
    select * 
    from agg.tidy_data_final
    where site = 20
"""

In [35]:
df = pd.read_sql_query(query, conn)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [36]:
df = df.set_index('timestamp', drop=False)
df = df.sort_index()

# Prepare Data Set for LSTM/GRU Models

In [37]:
# Parameters

#n_input = no. of time steps considered before making the prediction
#n_features = no. of variables/features considered
#n_output = no. of time steps to forecast

# Note: 1 time step = 30 mins
# Look at the last two weeks to forecast the next 36 hours

# 1 day = 2 * 24 * 1 = 24
# 2 hours = 2 * 2 = 8 

n_input = 144
n_features = 8
n_output = 12


# Select which features are to be inputted into the LSTM i.e., which columns in the df
df_input = df[['net_load', 'year', 'month', 'day', 'hour', 'day_of_week', 'season','precipitation_probability',
               'solar_radiation','sunshine_duration','weekend_or_bank_holiday','avg_net_load']]

# Normalise Data (as seen in Literature)

Group decision to use MaxAbs Scaler to normalise the data.

In [38]:
#Normalise all continuous variables needing scaling
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

variables = ['net_load','solar_radiation','sunshine_duration','avg_net_load']
for var in variables:
    new_var_name = var + '_norm'
    df_input[new_var_name] = scaler.fit_transform(df_input[[var]])

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be

Now, we also want to include the predictions for 'solar radiation', 'sunshine_duration', 'precipitation_probability' as inputs.
These will be treated as new inputs (t+48) ~ 48 half-hourly timestamps representing day-ahead predictions.

In [39]:
df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
df_input['precipitation_probability(t+48)'] = df_input['precipitation_probability'].shift(-48) #Add prediction for time the day after

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/3763762080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/3763762080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n

In [40]:
#We want to drop last 48 rows as they will have Na values for the (t+48) columns/inputs.
df_input.dropna(inplace=True)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_20951/1212278532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input.dropna(inplace=True)


# Generate X and Y Matrices

In [41]:
X, y = [], []
for i in range(len(df_input) - n_input - n_output + 1):
    # Select all columns for the input sequence
    X.append(df_input.iloc[i:(i + n_input)].values)  
    
    y.append(df_input['net_load'].iloc[(i + n_input):(i + n_input + n_output)].values)

X = np.array(X).reshape((len(X), n_input, -1))  
y = np.array(y)

# Train and Test Split

In [44]:
# Split into train (First ~330 days) and test (30 days)
# 1 time step = 30 mins

n_total_days = len(X) / (2 * 24)
n_test_days = 30
n_train_split = int((n_total_days - n_test_days) * 2 * 24)

X_train = X[:n_train_split]
y_train = y[:n_train_split]

X_test = X[n_train_split:]
y_test = y[n_train_split:]

# LSTM/GRU Sample Model